# Grabbing Live Telemetry Data

The actual telemetry is unavailable, other than `kms` (??) and GPS co-ordinates.

The times feed regularly updates and we should probably capture the most recent (so upsert).

The data feed is a time series, so each record needs its own entry. We can then generate a time series for each driver around their data samples.

There seem to be a couple of live feeds we can pull from:

In [28]:
url='https://webappsdata.wrc.com/srv/wrc/json/api/liveservice/getData?timeout=500'

import requests

livetimes = requests.get(url)
livetimes.json()['_entries'][12]

{'name': '025',
 'lon': 6.29777,
 'lat': 44.45213,
 'speed': 39.0,
 'heading': 136.0,
 'utx': 1579952440000,
 'driverid': '',
 'track': '',
 'status': 'Competing',
 'gear': 0,
 'throttle': 0,
 'brk': 0.0,
 'rpm': 0,
 'accx': 0.0,
 'accy': 0.0,
 'kms': 59.2,
 'altitude': 0}

In [35]:
from pandas.io.json import json_normalize
df = json_normalize(livetimes.json(), '_entries', meta='timestamp')
df.head()

,name,lon,lat,speed,heading,utx,driverid,track,status,gear,throttle,brk,rpm,accx,accy,kms,altitude,timestamp
0,003,6.06241,44.54491,3.0,222.0,1579952443000,,,Competing,0,0,0.0,0,0.0,0.0,49.3,0,1579952445000
1,004,6.10126,44.47804,81.0,272.0,1579952444500,,,Competing,0,0,0.0,0,0.0,0.0,38.4,0,1579952445000
2,008,6.60378,44.67164,0.0,120.0,1579866649500,,,Withdrawn,0,0,0.0,0,0.0,0.0,9.2,0,1579952445000
3,009,6.13025,44.47060,85.0,286.0,1579952441100,,,Competing,0,0,0.0,0,0.0,0.0,35.9,0,1579952445000
4,011,6.15421,44.46786,73.0,272.0,1579952439500,,,Competing,0,0,0.0,0,0.0,0.0,33.8,0,1579952445000


In [39]:
df.head(2).to_dict(orient='records')

[{'name': '003',
  'lon': 6.06241,
  'lat': 44.54491,
  'speed': 3.0,
  'heading': 222.0,
  'utx': 1579952443000,
  'driverid': '',
  'track': '',
  'status': 'Competing',
  'gear': 0,
  'throttle': 0,
  'brk': 0.0,
  'rpm': 0,
  'accx': 0.0,
  'accy': 0.0,
  'kms': 49.3,
  'altitude': 0,
  'timestamp': 1579952445000},
 {'name': '004',
  'lon': 6.10126,
  'lat': 44.47804,
  'speed': 81.0,
  'heading': 272.0,
  'utx': 1579952444500,
  'driverid': '',
  'track': '',
  'status': 'Competing',
  'gear': 0,
  'throttle': 0,
  'brk': 0.0,
  'rpm': 0,
  'accx': 0.0,
  'accy': 0.0,
  'kms': 38.4,
  'altitude': 0,
  'timestamp': 1579952445000}]

In [69]:
from sqlite_utils import Database

db = Database("testlive-wrc-mc.db")

In [44]:
test1 = db["test1"]
test1.upsert_all(df.to_dict(orient='records'),pk=['name','utx'])

<Table test1 (name, lon, lat, speed, heading, utx, driverid, track, status, gear, throttle, brk, rpm, accx, accy, kms, altitude, timestamp)>

In [55]:
import os
import json

def dir_loader(dbtable, path, suffix='txt'):
    for fn in [f for f in os.listdir(path) if f.split('.')[-1]==suffix]:
        with open(os.path.join(path,fn)) as f:
            _df = json_normalize( json.load(f) )
            dbtable.upsert_all(df.to_dict(orient='records'),
                               pk=['name','utx'])

In [56]:
dir_loader(db["test1"],'times/raw')

In [58]:
import pandas as pd
pd.read_sql('select * from test1 limit 3',db.conn)

,name,lon,lat,speed,heading,utx,driverid,track,status,gear,throttle,brk,rpm,accx,accy,kms,altitude,timestamp
0,003,6.06241,44.54491,3.0,222.0,1579952443000,,,Competing,0,0,0.0,0,0.0,0.0,49.3,0,1579952445000
1,004,6.10126,44.47804,81.0,272.0,1579952444500,,,Competing,0,0,0.0,0,0.0,0.0,38.4,0,1579952445000
2,008,6.60378,44.67164,0.0,120.0,1579866649500,,,Withdrawn,0,0,0.0,0,0.0,0.0,9.2,0,1579952445000


In [60]:
!ls -al testlive-wrc-mc-livegrab.db

-rw-r--r--  1 tonyhirst  staff  57344 25 Jan 13:01 testlive-wrc-mc-livegrab.db


In [63]:
!ls *xml

montecarlo_2020.xml


In [67]:
import kml2geojson
!mkdir -p gj_out
kml2geojson.main.convert('montecarlo_2020.xml','gj_out')


In [68]:
!ls gj_out

montecarlo_2020.geojson


Things like
https://webappsdata.wrc.com/srv/wrc/json/api/liveservice/getTimes?sitsid=%2283%22&stageid=%221122%22&stagename=%22SS12%22&timeout=5000
give start times but we can't tell which car is which?
The `startnumber` is perhaps one of the driver identifying values?

```
{
sitsid: "83",
stageid: "1122",
stagename: "SS12",
status: "",
_entries: [
{
startnumber: "1",
position: 4,
starttime: "16:44",
racetime: "10:11.6",
totaltime: "02:41:35.8",
totalpos: "2",
status: "COMPLETED",
_items: [
{
timestamp: 185009
},
{
timestamp: 338700
},
{
timestamp: 528800
}
]
},
{
startnumber: "10",
position: 0,
starttime: "16:41",
racetime: "",
totaltime: "",
totalpos: "",
status: "RUNNING",
_items: [
{
timestamp: 183400
}
]
},
```

Not sure whether anything in https://webappsdata.wrc.com/srv/wrc/json/api/liveservice/getData?timeout=5000 matches across to getTimes?

```
{
hash: "636932102363236630",
timestamp: 1557599034200,
_entries: [
{
name: "001",
lon: -73.07338,
lat: -36.77735,
speed: 0,
heading: 234,
utx: 1557599022300,
driverid: "",
track: "",
status: "Competing",
gear: 0,
throttle: 0,
brk: 0,
rpm: 0,
accx: 0,
accy: 0,
kms: 0.1,
altitude: 0
},
{
name: "003",
lon: -73.07195,
lat: -36.77664,
speed: 0,
heading: 254,
utx: 1557598210500,
driverid: "",
track: "",
status: "Competing",
gear: 0,
throttle: 0,
brk: 0,
rpm: 0,
accx: 0,
accy: 0,
kms: 0.3,
altitude: 0
},
...
]
```
The `name` is perhaps an identifying number.

In [8]:
url='https://www.wrc.com/live-ticker/live_popup_text.html?absolute=true'
r_html = requests.get(url)

In [9]:
r_html.text

'<!DOCTYPE html><html lang="de"><head><meta charset="utf-8">\n        <title>WRC Live Text</title><meta http-equiv=refresh content=\'60\'>\n        <meta name="description" content="WRC">\n\t\t<meta name="keywords" content="WRC,wrc.com">\n\t\t<meta name="news_keywords" content="WRC,wrc.com">\n\t\t<meta name="robots" content="index,follow">\n\t\t<meta name="original-source" content="https://www.wrc.com%2Fde%2F">\n        <meta name="viewport" content="width=device-width">\n        <meta property="og:title" content="WRC - wrc.com">\n\t\t<meta property="og:description" content="WRC">\n\t\t<meta property="og:image" content="">\n        <link href="/favicon.ico" type="image/x-icon" rel="shortcut icon">\n        <link media="screen" href="/style/main_wrc.css?1475580417" rel="stylesheet" type="text/css">\n        <link media="print" href="/style/print.css" rel="stylesheet" type="text/css">\n        <script src="/js/combined.js?v5" type="text/javascript"></script>\n        <!--[if lt IE 10]><s

In [24]:
import pandas as pd

url='https://webappsdata.wrc.com/srv/wrc/json/api/liveservice/getTimes?sitsid=%2283%22&stageid=%221122%22&stagename=%22SS12%22&timeout=5000'
url='https://webappsdata.wrc.com/srv/wrc/json/api/liveservice/getTimes?sitsid="83"&stageid="1122"&stagename="SS12"&timeout=5000'
livetimes = requests.get(url)
df_livetimes = pd.DataFrame(livetimes.json())
df_livetimes.head()

,sitsid,stageid,stagename,status,_entries
0,83,1122,SS12,INTERRUPTED,"{'startnumber': '1', 'position': 4, 'starttime..."
1,83,1122,SS12,INTERRUPTED,"{'startnumber': '10', 'position': 0, 'starttim..."
2,83,1122,SS12,INTERRUPTED,"{'startnumber': '100', 'position': 0, 'startti..."
3,83,1122,SS12,INTERRUPTED,"{'startnumber': '11', 'position': 0, 'starttim..."
4,83,1122,SS12,INTERRUPTED,"{'startnumber': '19', 'position': 1, 'starttim..."


In [27]:
from pandas.io.json import json_normalize
json_normalize(livetimes.json(),['_entries', '_items'])#['_items'].apply(pd.Series)
#startnumber

,timestamp
0,185009
1,338700
2,528800
3,183400
4,179700
...,...
116,725519
117,1082815
118,394463
119,741112


In [25]:
json_normalize(livetimes.json(),['_entries'])#['_items'].apply(pd.Series)


,startnumber,position,starttime,racetime,totaltime,totalpos,status,_items
0,1,4,16:44,10:11.6,02:41:35.8,2,COMPLETED,"[{'timestamp': 185009}, {'timestamp': 338700},..."
1,10,0,16:41,,,,DNS,[{'timestamp': 183400}]
2,100,0,,,,,,[]
3,11,0,,,,,DNS,[]
4,19,1,16:35,10:00.6,02:41:40.9,3,COMPLETED,"[{'timestamp': 179700}, {'timestamp': 327800},..."
...,...,...,...,...,...,...,...,...
58,95,31,17:31,12:31.6,03:22:01.5,27,COMPLETED,"[{'timestamp': 348200}, {'timestamp': 641181},..."
59,96,0,,,,,,[]
60,97,38,17:41,12:49.1,,,RUNNING,"[{'timestamp': 383998}, {'timestamp': 725519},..."
61,98,0,,,,,,[]


In [26]:
#zz = pd.concat([df_livetimes.drop(['_entries'], axis=1), df_livetimes['_entries'].apply(pd.Series)], axis=1)
zz=df_livetimes['_entries'].apply(pd.Series)
zz.head()

,startnumber,position,starttime,racetime,totaltime,totalpos,status,_items
0,1,4,16:44,10:11.6,02:41:35.8,2,COMPLETED,"[{'timestamp': 185009}, {'timestamp': 338700},..."
1,10,0,16:41,,,,RUNNING,[{'timestamp': 183400}]
2,100,0,,,,,,[]
3,11,0,,,,,DNS,[]
4,19,1,16:35,10:00.6,02:41:40.9,3,COMPLETED,"[{'timestamp': 179700}, {'timestamp': 327800},..."


In [23]:
zz.iloc[0]['_items']

[{'timestamp': 185009}, {'timestamp': 338700}, {'timestamp': 528800}]

In [27]:
#COMPLETED', 'RUNNING', '', 'DNS', 'TORUN'
zz['status'].unique()

array(['COMPLETED', 'RUNNING', '', 'DNS', 'TORUN'], dtype=object)

In [28]:
df_livetimes['status'].unique()
#RUNNING

array(['RUNNING'], dtype=object)

In [30]:
url='https://webappsdata.wrc.com/srv/wrc/json/api/liveservice/getTimes?sitsid="83"'
requests.get(url).json()
'''
{'sitsid': '83',
 'stageid': None,
 'stagename': None,
 'status': '',
 '_entries': []}
'''
#So we need to pass  in the sitsid, stageid and stagename to get the data back

{'sitsid': '83',
 'stageid': None,
 'stagename': None,
 'status': '',
 '_entries': []}